In [5]:
import json
import pandas as pd

# Step 1: Load both JSON files
with open('./roland_garros_2025_round1.json', 'r', encoding='utf-8') as f1:
    data_r1 = json.load(f1)

with open('./roland_garros_2025_round2.json', 'r', encoding='utf-8') as f2:
    data_r2 = json.load(f2)

# Step 2: Load the player ID CSV
df_players = pd.read_csv('./PLAYERS_ID.csv')

# Step 3: Build normalized name → ID dictionary
def make_full_name(row):
    return f"{row['last_name'].strip().upper()} {row['first_name'].strip()}"

df_players['full_name'] = df_players.apply(make_full_name, axis=1)
name_to_id = dict(zip(df_players['full_name'], df_players['id']))

# Just for debug
print("Sample keys in name_to_id:")
for name in list(name_to_id.keys())[:5]:
    print(" →", name)

# Step 4: Normalize player name exactly like in the dict keys
def normalize_player_name(name):
    parts = name.strip().split(' ', 1)
    if len(parts) != 2:
        return name.strip().upper()  # fallback
    last = parts[0].upper()
    first = parts[1].strip()
    return f"{last} {first}"

# Step 5: Assign IDs using normalized name
def assign_ids(match):
    for key in ['player1', 'player2']:
        raw_name = match[key]['name']
        norm_name = normalize_player_name(raw_name)
        match[key]['id'] = name_to_id.get(norm_name)
        if match[key]['id'] is None:
            print(f"[Warning] Player not found: '{raw_name}' → '{norm_name}'")
    return match

# Step 6: Process matches
matches_r1 = [assign_ids(match) for match in data_r1['matches']]
matches_r2 = [assign_ids(match) for match in data_r2['matches']]
all_matches = matches_r1 + matches_r2

# Step 7: Combine and save
combined_data = {
    "tournament": "Roland-Garros 2025",
    "surface": "CLAY",
    "matches": all_matches
}

with open('./roland_garros_2025_complete_final.json', 'w', encoding='utf-8') as f:
    json.dump(combined_data, f, ensure_ascii=False, indent=2)

print("\n✅ Final JSON saved to: roland_garros_2025_complete_final.json")

Sample keys in name_to_id:
 → ALCARAZ Carlos
 → ALTMAIER Daniel
 → ARNALDI Matteo
 → ATMANE Terence
 → AUGER-ALIASSIME FELIX
[Warning] Player not found: 'HERBERT Pierre-Hugues' → 'HERBERT Pierre-Hugues'
[Warning] Player not found: 'AUGER-ALIASSIME Felix' → 'AUGER-ALIASSIME Felix'
[Warning] Player not found: 'GOMEZ Federico' → 'GOMEZ Federico'
[Warning] Player not found: 'O'CONNELL Christopher' → 'O'CONNELL Christopher'
[Warning] Player not found: 'BU Yunchaokete' → 'BU Yunchaokete'
[Warning] Player not found: 'RAMOS-VINOLAS Albert' → 'RAMOS-VINOLAS Albert'
[Warning] Player not found: 'STRUFF Jan-Lennard' → 'STRUFF Jan-Lennard'
[Warning] Player not found: 'MENŠÍK Jakub' → 'MENŠÍK Jakub'
[Warning] Player not found: 'GOMEZ Federico' → 'GOMEZ Federico'
[Warning] Player not found: 'JANU Filip Cristian' → 'JANU Filip Cristian'
[Warning] Player not found: 'KLEIN Lukas' → 'KLEIN Lukas'
[Warning] Player not found: 'DE GALAN Daniel Elahi' → 'DE GALAN Daniel Elahi'
[Warning] Player not found: 'PO